In [12]:
import torch
import torchvision
import torchvision.transforms as transforms

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

def get_vis_loader():
    transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                              shuffle=True, num_workers=2)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                             shuffle=False, num_workers=2)
    return trainloader, testloader

def get_train_loader(batch_size, transform):
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    return trainloader
    
def get_test_loader(batch_size, transform):
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=2)
    return testloader
    
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

vistrainloader, vistestloader = get_vis_loader()
trainloader = get_train_loader(batch_size, transform)
testloader = get_test_loader(batch_size, transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [15]:
import torch.nn as nn
import torch.nn.functional as F


class ConvolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=2, stride=2)
        
        self.conv3 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        
        self.conv4 = nn.Conv2d(32, 64, kernel_size=5, padding=2, stride=2)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 120)
        
        self.fc2 = nn.Linear(120, 84)
        
        self.fc3 = nn.Linear(84, 10)
        

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.relu(self.conv5(x))
        
        x = self.maxpool(x)
        
        x = x.view(-1, 128 * 4 * 4)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

cnn_net = ConvolutionNet()

In [16]:
import torch.optim as optim

def get_optimizer(net, lr):
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    return optimizer

def accuracy(output, target):
    # get the index of the max log-probability
    pred = output.max(1, keepdim=True)[1]
    return pred.eq(target.view_as(pred)).float().mean()

criterion = nn.CrossEntropyLoss()
optimizer = get_optimizer(fc_net, 0.01)

In [17]:
def forward_step(net, inputs, labels):
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    return outputs, loss, labels


def train(net, loader, optimizer, max_epoch):
    net.to(device)
    net.train()
    N = len(loader)
    print_interval = (N // 8 // 100 + 1) * 100
    for epoch in range(max_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        running_acc = 0.0
        for i, data in enumerate(loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            images, labels = data[0].to(device), data[1].to(device) 

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs, loss, labels = forward_step(net, images, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            running_acc += accuracy(outputs, labels).item()
            if (i + 1) % print_interval == 0:
                print('Epoch: [%d / %d], batches: [%d / %d], loss: %.3f, acc: %.2f' %
                      (epoch + 1, max_epoch, i + 1, N, 
                       running_loss / print_interval, 100 * running_acc / print_interval))
                running_loss = 0.0
                running_acc = 0.0

    print('Finished Training')

In [18]:
optimizer_cnn = get_optimizer(cnn_net, 0.01)
train(cnn_net, trainloader, optimizer_cnn, 5)

Epoch: [1 / 5], batches: [100 / 782], loss: 2.303, acc: 10.56
Epoch: [1 / 5], batches: [200 / 782], loss: 2.303, acc: 9.88
Epoch: [1 / 5], batches: [300 / 782], loss: 2.303, acc: 10.20
Epoch: [1 / 5], batches: [400 / 782], loss: 2.300, acc: 12.47
Epoch: [1 / 5], batches: [500 / 782], loss: 2.228, acc: 18.22
Epoch: [1 / 5], batches: [600 / 782], loss: 2.067, acc: 22.64
Epoch: [1 / 5], batches: [700 / 782], loss: 1.969, acc: 27.39
Epoch: [2 / 5], batches: [100 / 782], loss: 1.777, acc: 34.91
Epoch: [2 / 5], batches: [200 / 782], loss: 1.673, acc: 38.89
Epoch: [2 / 5], batches: [300 / 782], loss: 1.626, acc: 39.88
Epoch: [2 / 5], batches: [400 / 782], loss: 1.557, acc: 42.94
Epoch: [2 / 5], batches: [500 / 782], loss: 1.543, acc: 44.28
Epoch: [2 / 5], batches: [600 / 782], loss: 1.489, acc: 45.23
Epoch: [2 / 5], batches: [700 / 782], loss: 1.448, acc: 47.66
Epoch: [3 / 5], batches: [100 / 782], loss: 1.361, acc: 49.92
Epoch: [3 / 5], batches: [200 / 782], loss: 1.342, acc: 51.61
Epoch: [3

In [19]:
def test(net, testloader):
    correct = 0
    total = 0
    net.to(device)
    net.eval()
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

In [20]:
test(cnn_net, testloader)

Accuracy of the network on the 10000 test images: 61 %
